<a href="https://colab.research.google.com/github/Jahan08/hERG-web-app/blob/main/Deepneuralnetwork_pytorch_hERG_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install torch skorch scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 3.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from skorch import NeuralNetBinaryClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

In [57]:
df = pd.read_csv("/content/hERG_bioactivity_pIC50.csv",header=None)
df

,0,1,2,3,4,5,6,7
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,0.0140,Yes,7.853872
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,0.0322,Yes,7.492144
2,CHEMBL691014,K+ channel blocking activity in human embryoni...,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,J Med Chem,CHEMBL6966,0.1430,Yes,6.844664
3,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,J Med Chem,CHEMBL1107,0.1960,Yes,6.707744
4,CHEMBL691014,K+ channel blocking activity in human embryoni...,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,J Med Chem,CHEMBL998,0.1730,Yes,6.761954
...,...,...,...,...,...,...,...,...
2963,CHEMBL5048865,Inhibition of hERG by patch clamp method,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,J Med Chem,CHEMBL5081517,44.0000,No,4.356547
2964,CHEMBL5048865,Inhibition of hERG by patch clamp method,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,J Med Chem,CHEMBL5072442,33.0000,No,4.481486
2965,CHEMBL5049389,Inhibition of hERG,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,J Med Chem,CHEMBL5093378,0.6700,Yes,6.173925
2966,CHEMBL5050750,Inhibition of human ERG,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,ACS Med Chem Lett,CHEMBL5070887,0.0140,Yes,7.853872


In [58]:
y = df.iloc[:,-2]
y

0       Yes
1       Yes
2       Yes
3       Yes
4       Yes
       ... 
2963     No
2964     No
2965    Yes
2966    Yes
2967     No
Name: 6, Length: 2968, dtype: object

In [59]:
X = pd.read_csv("/content/Extended_descriptors_list.csv", header=None)
X

,0,1,2,3,4,5,6,7,8,9,...,958,959,960,961,962,963,964,965,966,967
0,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,1,1,0,0,0,0,1,1,...,0,0,1,0,0,1,1,1,1,0
2,0,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,1,0,0,0,0,0,0,1,...,0,1,0,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2964,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2965,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2966,1,1,1,1,0,1,0,1,0,1,...,1,0,0,1,0,1,0,1,1,0


###NeuralNetBinaryClassifier

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
X_train.shape, y_train.shape

((2374, 968), (2374,))

In [48]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)

# Convert to 2D PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

In [49]:
class hERGClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(968,968)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(968, 968)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(968, 968)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(968, 1)

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.output(x)
        return x

In [50]:
# create the skorch wrapper
model = NeuralNetBinaryClassifier(
    hERGClassifier,
    criterion=torch.nn.BCEWithLogitsLoss,
    optimizer=torch.optim.Adam,
    lr=0.0001,
    max_epochs=150,
    batch_size=10,
    verbose=False
)

In [51]:
X_train.shape, y_train.shape

(torch.Size([2374, 968]), torch.Size([2374]))

This NeuralNetBinaryClassifier instance is not initialized yet. Call 'initialize' or 'fit'

In [52]:
model.fit(X_train, y_train)

<class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=hERGClassifier(
    (layer1): Linear(in_features=968, out_features=968, bias=True)
    (act1): ReLU()
    (layer2): Linear(in_features=968, out_features=968, bias=True)
    (act2): ReLU()
    (layer3): Linear(in_features=968, out_features=968, bias=True)
    (act3): ReLU()
    (output): Linear(in_features=968, out_features=1, bias=True)
  ),
)

In [61]:
y_train_pred = model.predict(X_train)

TypeError: ignored

In [55]:
y_test_pred = model.predict(X_test)

TypeError: ignored

In [20]:
from sklearn.metrics import matthews_corrcoef

In [56]:
mcc_train = matthews_corrcoef(y_train, y_train_pred)
mcc_train

0.9410825209794222

In [ ]:
mcc_test = matthews_corrcoef(y_test, y_test_pred)
mcc_test

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(model, X, y, cv=kfold)
print("mean = %.3f; std = %.3f" % (results.mean(), results.std()))

mean = 0.859; std = 0.022


In [ ]:

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
#prediction probability
r_probs_train = [0 for _ in range(len(y_train))]
model_probs_train = model.predict_proba(X_train)
#nb_probs = nb.predict_proba(X_testscaled)

In [ ]:
model_probs_train = model_probs_train[:, 1]
model_probs_train

In [ ]:
r_fpr_train, r_tpr_train, _ = roc_curve(y_train, r_probs_train, pos_label=2)
model_fpr_train, model_tpr_train, _ = roc_curve(y_train, model_probs_train, pos_label=2)

In [ ]:
import pickle
pickle.dump(model, open('hERG_model_NNBinaryClassifier.pkl', 'wb'))

In [ ]:
! pip3 install streamlit

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

"model.predict" refers to a method in machine learning where a trained model is used to make predictions on new data. The "model" refers to the machine learning algorithm that has been trained on a dataset, and "predict" is the function used to make predictions on new data. In order to use this method, the input data must be preprocessed in the same way as the training data, and the output of the prediction will depend on the type of model being used (e.g. regression for continuous output, classification for categorical output).

In [62]:
model.predict(X)

TypeError: ignored

method called "predict_proba" in a machine learning model. "predict_proba" is a method used to predict the probability of a certain outcome or class in a classification problem. It is commonly used in supervised learning algorithms such as logistic regression, decision trees, and random forests. The output of this method is a probability score between 0 and 1 for each class, indicating the likelihood of the input belonging to that class.

In [ ]:
model.predict_proba(X)

array([[3.3378601e-06, 9.9999666e-01],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       ...,
       [4.7683716e-07, 9.9999952e-01],
       [0.0000000e+00, 1.0000000e+00],
       [9.9999881e-01, 1.1710529e-06]], dtype=float32)

###MLPclassifier

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
import numpy as np

# binary encoding of labels
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

# create model
model = MLPClassifier(hidden_layer_sizes=(60,60,60), activation='relu',
                      max_iter=150, batch_size=10, verbose=False)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(model, X, y, cv=kfold)
print("mean = %.3f; std = %.3f" % (results.mean(), results.std()))